## Import Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
file_path = 'C:/Users/Demi/Desktop/TB2/Mini project/simulated_transaction_2024.csv'
data = pd.read_csv(file_path)
data

,Date,Timestamp,Account No,Balance,Amount,Third Party Account No,Third Party Name
0,01/01/2023,00:00,678330503.0,2971.000000,1584.00,NaN,Westport Care Home
1,01/01/2023,00:00,472213568.0,3792.000000,1950.00,NaN,Barbiee Boutique
2,01/01/2023,00:00,472213568.0,3012.000000,-780.00,283027736.0,NaN
3,01/01/2023,00:00,283027736.0,1787.000000,780.00,472213568.0,NaN
4,01/01/2023,00:00,624500124.0,3226.000000,1825.00,NaN,Fat Face
...,...,...,...,...,...,...,...
230591,06/12/2023,20:54,581655972.0,45935.206861,-41.06,NaN,Tesco
230592,06/12/2023,20:55,786141370.0,-244.837500,-62.35,NaN,Sainsbury Local
230593,06/12/2023,21:05,824916823.0,9709.172159,-32.94,NaN,Deliveroo
230594,06/12/2023,21:13,366550080.0,26834.165794,-19.25,NaN,Amazon


In [3]:
data.isnull().sum()

Date                         229
Timestamp                    251
Account No                   224
Balance                      246
Amount                       209
Third Party Account No    223764
Third Party Name            7079
dtype: int64

# Handling missing data in Date and Timestamp 

In [4]:
data[data['Date'].isna() & data['Timestamp'].isna()]

,Date,Timestamp,Account No,Balance,Amount,Third Party Account No,Third Party Name


##### No row have missing data in 'Date' and 'Timestamp' at same time

In [5]:
data[data['Date'].isna() | data['Timestamp'].isna()]

,Date,Timestamp,Account No,Balance,Amount,Third Party Account No,Third Party Name
2734,01/01/2023,NaN,593191200.0,1490.020000,-14.99,NaN,Blizzard
3918,NaN,00:00,554792076.0,575.020000,-5.99,NaN,Mojang Studios
4106,NaN,00:00,581655972.0,6895.010000,-15.99,NaN,Netflix
4511,NaN,00:00,711140907.0,1217.000000,-7.00,NaN,Xbox
4710,NaN,09:23,310620484.0,992.080000,-11.98,NaN,The Works
...,...,...,...,...,...,...,...
228485,03/12/2023,NaN,165691047.0,793.947100,-67.97,NaN,Matalan
228813,04/12/2023,NaN,287230540.0,997.120998,-660.00,447298754.0,NaN
229139,04/12/2023,NaN,727417797.0,5970.393627,-39.77,NaN,Fat Face
230045,NaN,19:52,196289502.0,18280.425041,-49.93,NaN,Etsy


In [6]:
df = data.copy()

In [7]:
df['Timestamp_delta'] = pd.to_timedelta(df['Timestamp'] + ':00')

# Perform forward and backward filling for Dates
df['Date_forward'] = df['Date'].ffill()
df['Date_backward'] = df['Date'].bfill()
# Perform forward filling for Timestamps
df['Timestamp_forward'] = df['Timestamp_delta'].ffill()

# Creating a temporary column for comparison.
df['Prev_Timestamp'] = df['Timestamp_delta'].shift(1)
df['Next_Timestamp'] = df['Timestamp_delta'].shift(-1)
df['Prev_Date'] = df['Date'].shift(1)
df['Next_Date'] = df['Date'].shift(-1)


# Handling missing values in Date
for i in range(len(df)):
    if pd.isnull(df.loc[i, 'Date']):
        # If forward and backward Dates are the same
        if df.loc[i, 'Date_forward'] == df.loc[i, 'Date_backward']:
            df.loc[i, 'Date'] = df.loc[i, 'Date_forward']
        else:
            # If Dates are different, compare Timestamps to determine the correct Date
            if df.loc[i, 'Timestamp_delta'] >= df.loc[i, 'Prev_Timestamp']: # 23:08 & 23:14, still same day
                df.loc[i, 'Date'] = df.loc[i, 'Prev_Date']
            else:
                df.loc[i, 'Date'] = df.loc[i, 'Next_Date']  # 23:08 & 00:14, start a new day

# Handling missing values in Timestamps
for i in range(len(df)):
    if pd.isnull(df.loc[i, 'Timestamp']):
        # If it has same Date with the previous entry, use the previous entry's Timestamp for filling
        if df.loc[i, 'Date'] == df.loc[i, 'Prev_Date']:
            df.loc[i, 'Timestamp'] = df.loc[i-1, 'Timestamp']
        else:
            df.loc[i, 'Timestamp'] = df.loc[i+1, 'Timestamp']


df.drop(['Timestamp_delta', 'Date_forward', 'Date_backward', 'Timestamp_forward', 'Prev_Timestamp', 'Next_Timestamp', 'Prev_Date', 'Next_Date'], axis=1, inplace=True)
df.head()

,Date,Timestamp,Account No,Balance,Amount,Third Party Account No,Third Party Name
0,01/01/2023,00:00,678330503.0,2971.0,1584.0,NaN,Westport Care Home
1,01/01/2023,00:00,472213568.0,3792.0,1950.0,NaN,Barbiee Boutique
2,01/01/2023,00:00,472213568.0,3012.0,-780.0,283027736.0,NaN
3,01/01/2023,00:00,283027736.0,1787.0,780.0,472213568.0,NaN
4,01/01/2023,00:00,624500124.0,3226.0,1825.0,NaN,Fat Face


In [8]:
df.isnull().sum()

Date                           0
Timestamp                      0
Account No                   224
Balance                      246
Amount                       209
Third Party Account No    223764
Third Party Name            7079
dtype: int64

In [9]:
# Combine Data and Timestamp
from datetime import datetime

df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Timestamp'], format='%d/%m/%Y %H:%M').dt.strftime('%Y-%m-%d %H:%M')
df = df.drop(['Date', 'Timestamp'], axis=1)

df

,Account No,Balance,Amount,Third Party Account No,Third Party Name,DateTime
0,678330503.0,2971.000000,1584.00,NaN,Westport Care Home,2023-01-01 00:00
1,472213568.0,3792.000000,1950.00,NaN,Barbiee Boutique,2023-01-01 00:00
2,472213568.0,3012.000000,-780.00,283027736.0,NaN,2023-01-01 00:00
3,283027736.0,1787.000000,780.00,472213568.0,NaN,2023-01-01 00:00
4,624500124.0,3226.000000,1825.00,NaN,Fat Face,2023-01-01 00:00
...,...,...,...,...,...,...
230591,581655972.0,45935.206861,-41.06,NaN,Tesco,2023-12-06 20:54
230592,786141370.0,-244.837500,-62.35,NaN,Sainsbury Local,2023-12-06 20:55
230593,824916823.0,9709.172159,-32.94,NaN,Deliveroo,2023-12-06 21:05
230594,366550080.0,26834.165794,-19.25,NaN,Amazon,2023-12-06 21:13


# Handling missing data in Balance and Amount

## check

In [10]:
df[df['Balance'].isna() | df['Amount'].isna()]

,Account No,Balance,Amount,Third Party Account No,Third Party Name,DateTime
949,857754342.0,2745.750000,NaN,NaN,LBG,2023-01-01 00:00
1392,246753533.0,NaN,-100.00,NaN,Grand Union BJJ,2023-01-01 00:00
1821,424397946.0,3358.010000,NaN,NaN,PureGym,2023-01-01 00:00
3937,987656636.0,844.020000,NaN,NaN,Disney,2023-01-01 00:00
4071,399538448.0,3506.070000,NaN,NaN,Amazon,2023-01-01 00:00
...,...,...,...,...,...,...
226295,661295096.0,3882.600950,NaN,NaN,Disney,2023-11-30 23:59
227463,460463595.0,6.012055,NaN,NaN,Tesco,2023-12-01 20:04
227645,587225252.0,7342.350655,NaN,NaN,Sainsbury Local,2023-12-02 07:37
229764,117273481.0,NaN,-6.15,NaN,Starbucks,2023-12-05 09:17


In [11]:
df[df['Balance'].isna() & df['Amount'].isna()]

,Account No,Balance,Amount,Third Party Account No,Third Party Name,DateTime


##### No row have missing data in 'Balance' and 'Amount' at same time
##### Because we need to calculate missing data according to Account No

In [12]:
df[df['Balance'].isna() & df['Account No'].isna()]

,Account No,Balance,Amount,Third Party Account No,Third Party Name,DateTime
163710,NaN,NaN,-19.19,NaN,Deliveroo,2023-09-05 18:22


In [13]:
df[df['Amount'].isna() & df['Account No'].isna()]

,Account No,Balance,Amount,Third Party Account No,Third Party Name,DateTime


## Calculate missing 'Balance'
####  In case of missing record, use two methods: balance and amout in previous and behind to calculate balance. If the difference is less than 1, then use it. If more than 1, use the closer one.
##### If Keep 2 decimal places and leave 76 NAs, 1 decimal place and leave 50 NAs, and the difference is less than 1 and leave 47 NAs.

In [14]:
df['Balance'].isnull().sum()

246

In [15]:
df.sort_values(by=['Account No', 'DateTime'], inplace=True)
df = df.reset_index(drop=False)
df['DateTime'] = pd.to_datetime(df['DateTime'], format='%Y-%m-%d %H:%M')

# Save index of missing Balance with uncontinuous record
#uncontinuous_index = []

for i in range(1, len(df)):
    if pd.isnull(df.iloc[i]['Balance']):
        calc_balance_1 = None
        calc_balance_2 = None
        #  In case of missing record, use two methods to calculate balance
        if df.iloc[i]['Account No'] == df.iloc[i-1]['Account No']: # Check if the current row and the previous row belong to the same account
            calc_balance_1 = df.iloc[i-1]['Balance'] + df.iloc[i]['Amount']
        if df.iloc[i]['Account No'] == df.iloc[i+1]['Account No']: # Check if the current row and the next row belong to the same account
            calc_balance_2 = df.iloc[i+1]['Balance'] - df.iloc[i+1]['Amount']

        if calc_balance_1 is not None and calc_balance_2 is not None:
            # When both methods give the same result, then fill the 'Balance'
            if abs(calc_balance_1 - calc_balance_2) <= 1:
                df.at[i, 'Balance'] = calc_balance_1    
            # When give unequal results, record is not continuous, choose the closer one in DateTime
            if abs(calc_balance_1 - calc_balance_2) > 1:
                #uncontinuous_index.extend([i-1, i, i+1]) 
                time_diff_prev = abs(df.iloc[i]['DateTime'] - df.iloc[i - 1]['DateTime'])
                time_diff_next = abs(df.iloc[i]['DateTime'] - df.iloc[i + 1]['DateTime'])
                if time_diff_prev < time_diff_next: 
                    df.at[i, 'Balance'] = calc_balance_1
                else:
                    df.at[i, 'Balance'] = calc_balance_2

# Check missing Balance with uncontinuous record
#uncontinuous_index.sort()
#uncontinuous = df.iloc[uncontinuous_index]
#uncontinuous.to_csv('uncontinuous.csv')
                
df['Balance'].isnull().sum()

1

In [16]:
df[df['Balance'].isna()]

,index,Account No,Balance,Amount,Third Party Account No,Third Party Name,DateTime
230519,163710,NaN,NaN,-19.19,NaN,Deliveroo,2023-09-05 18:22:00


One row cannot calculate because of missing 'Account No'

## Calculate missing 'Amount'

### fixed values

In [17]:
df['Amount'].isnull().sum()

209

##### Add assuming Amount from the difference between two Balances

In [18]:
for i in df[df['Amount'].isnull()].index:
    account_no = df.at[i, 'Account No']
    if df.at[i - 1, 'Account No'] == account_no:  # Ensure the previous record is from the same account
        calc_amount = df.at[i, 'Balance'] - df.at[i - 1, 'Balance']  
        df.at[i, 'Calculated_Amount'] = calc_amount
    else:
        continue

##### There are fixed deductions and disbursements such as salaries, loans, and monthly fees that occur on the first day of each month at 00:00, and on the last day of each month at both 00:00 and 23:59, totaling 47 instances.

In [19]:
missing_amount = df[df['Amount'].isna()]
start_end = missing_amount[missing_amount['DateTime'].dt.strftime('%H:%M').isin(['00:00', '23:59'])]
start_end = start_end.sort_values('DateTime', ascending=True)
#start_end = start_end[~start_end['Third Party Name'].isnull()]

print(start_end.shape[0])
start_end

54


,index,Account No,Balance,Amount,Third Party Account No,Third Party Name,DateTime,Calculated_Amount
226851,3937,987656636.0,844.020000,NaN,NaN,Disney,2023-01-01 00:00:00,-7.99
195445,949,857754342.0,2745.750000,NaN,NaN,LBG,2023-01-01 00:00:00,-1268.00
79963,1821,424397946.0,3358.010000,NaN,NaN,PureGym,2023-01-01 00:00:00,-18.99
72806,4071,399538448.0,3506.070000,NaN,NaN,Amazon,2023-01-01 00:00:00,-7.99
80375,20245,425730826.0,1349.167609,NaN,191631876.0,NaN,2023-01-30 23:59:00,1910.00
31757,23836,224112146.0,393.642650,NaN,NaN,Netflix,2023-01-31 23:59:00,-15.99
98541,22434,496395938.0,5197.963098,NaN,NaN,LBG,2023-02-01 00:00:00,-561.00
139396,22401,644751433.0,1114.552142,NaN,NaN,LBG,2023-02-01 00:00:00,-1752.00
192713,22111,851181128.0,4679.511779,NaN,NaN,Craftastic,2023-02-01 00:00:00,509.95
116354,42431,552980092.0,86.845719,NaN,NaN,Blizzard,2023-02-28 23:59:00,-14.99


### Fix values in Third Party Account No

In [20]:
df[df['Third Party Account No'].notnull() & df['Amount'].isnull()]

,index,Account No,Balance,Amount,Third Party Account No,Third Party Name,DateTime,Calculated_Amount
4157,94385,119993184.0,472.007697,NaN,384497904.0,NaN,2023-05-31 23:59:00,1282.00
19614,220964,175452235.0,-500.548250,NaN,835736339.0,NaN,2023-11-29 23:59:00,1310.00
58109,93265,331684336.0,543.955030,NaN,250957494.0,NaN,2023-05-30 23:59:00,1170.00
58834,155900,336557504.0,2583.640089,NaN,578131296.0,NaN,2023-08-31 23:59:00,2131.66
80375,20245,425730826.0,1349.167609,NaN,191631876.0,NaN,2023-01-30 23:59:00,1910.00
182247,200280,812012103.0,-781.904300,NaN,291580432.0,NaN,2023-10-31 23:59:00,620.00
189146,93257,834581602.0,1519.622150,NaN,444371478.0,NaN,2023-05-30 23:59:00,1532.00
207358,30751,899983008.0,-160.647450,NaN,791220575.0,NaN,2023-02-08 22:21:00,-200.00


#### Because most of missing in these rows are fix, use mode to fill

In [21]:
from scipy import stats

modes = df[df['Third Party Account No'].notnull() & df['Amount'].notnull()].groupby('Third Party Account No')['Amount'].agg(pd.Series.mode)
mode_dict = modes.to_dict()
df.loc[df['Amount'].isnull() & df['Third Party Account No'].notnull(), 'Amount'] = df['Third Party Account No'].map(mode_dict)

C:\Users\Demi\AppData\Local\Temp\ipykernel_9652\70339268.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-430.0 1310.0 -780.0 -710.0 1910.0 620.0 -510.0
 array([-300., -250., -210., -200., -150., -140., -110., -100.,  -80.,
         -50.])                                                       ]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['Amount'].isnull() & df['Third Party Account No'].notnull(), 'Amount'] = df['Third Party Account No'].map(mode_dict)


In [22]:
indices = [4157, 19614, 58109, 58834, 80375, 182247, 189146, 207358]
extracted_rows = df.loc[indices]
extracted_rows

,index,Account No,Balance,Amount,Third Party Account No,Third Party Name,DateTime,Calculated_Amount
4157,94385,119993184.0,472.007697,-430.0,384497904.0,NaN,2023-05-31 23:59:00,1282.00
19614,220964,175452235.0,-500.548250,1310.0,835736339.0,NaN,2023-11-29 23:59:00,1310.00
58109,93265,331684336.0,543.955030,-780.0,250957494.0,NaN,2023-05-30 23:59:00,1170.00
58834,155900,336557504.0,2583.640089,-710.0,578131296.0,NaN,2023-08-31 23:59:00,2131.66
80375,20245,425730826.0,1349.167609,1910.0,191631876.0,NaN,2023-01-30 23:59:00,1910.00
182247,200280,812012103.0,-781.904300,620.0,291580432.0,NaN,2023-10-31 23:59:00,620.00
189146,93257,834581602.0,1519.622150,-510.0,444371478.0,NaN,2023-05-30 23:59:00,1532.00
207358,30751,899983008.0,-160.647450,"[-300.0, -250.0, -210.0, -200.0, -150.0, -140....",791220575.0,NaN,2023-02-08 22:21:00,-200.00


In [23]:
# Mannual change Amount
df.at[207358, 'Amount'] = df.at[207358, 'Calculated_Amount']

#### this one is not fix, just use the Calculated result

### Fix values in Third Party Name

#### Find the Amount from the last instance and the next instance for the same time across different months. total of 38 instances are filled.
#### like 1/1 00:00 use 2/1 00:00 in same Account and Third Party, and 1/31 00:00 use 2/28 00:00, 2/28 23:59 use 3/31 23:59

In [24]:
df['DateTime'] = pd.to_datetime(df['DateTime'])

filled_indices = []
filled_indices2 = []

# calculate special data and timestamp
def get_special_dates(year, month):
    first_day_of_month = pd.Timestamp(year=year, month=month, day=1)
    last_day_of_month = first_day_of_month + pd.offsets.MonthEnd(0)
    last_day_of_month_0000 = last_day_of_month.replace(hour=00, minute=00, second=00)
    last_day_of_month_2359 = last_day_of_month.replace(hour=23, minute=59, second=00)
    return first_day_of_month, last_day_of_month_0000, last_day_of_month_2359

# Find Na of Amount
for i, row in df[df['Amount'].isnull()].iterrows():
    account_no = row['Account No']
    third_party_name = row['Third Party Name']
    current_datetime = row['DateTime']
    year, month = current_datetime.year, current_datetime.month
    
    first_day_of_month, last_day_of_month_0000, last_day_of_month_2359 = get_special_dates(year, month)
    
    # Compare data and timestamp 
    target_dates = []
    if current_datetime in [first_day_of_month, last_day_of_month_0000, last_day_of_month_2359]:
        if current_datetime == first_day_of_month:
            target_dates = [first_day_of_month - pd.offsets.MonthBegin(1), first_day_of_month + pd.offsets.MonthBegin(1)]
        elif current_datetime == last_day_of_month_0000:
            target_dates = [last_day_of_month_0000 - pd.offsets.MonthEnd(1), last_day_of_month_0000 + pd.offsets.MonthEnd(1)]
        elif current_datetime == last_day_of_month_2359:
            target_dates = [last_day_of_month_2359 - pd.offsets.MonthEnd(1), last_day_of_month_2359 + pd.offsets.MonthEnd(1)]
    
    # fill NA of Amount
    for td in target_dates:
        matching_record = df[(df['Account No'] == account_no) & 
                             (df['Third Party Name'] == third_party_name) & 
                             (df['DateTime'] == td)]
        if not matching_record.empty:
            fill_amount = matching_record.iloc[0]['Amount']
            df.at[i, 'Amount'] = fill_amount
            filled_indices.append(i)
            filled_indices2.append(df.at[i, 'index'])
            break  

print(f"Filled indices: {filled_indices}")
length_filled_indices = len(filled_indices)
length_filled_indices

Filled indices: [7254, 9623, 14837, 20032, 27689, 30462, 30901, 31757, 32626, 42532, 46882, 57271, 58598, 70983, 79628, 85646, 89220, 96880, 98505, 98541, 116354, 129831, 135000, 136126, 139396, 140223, 142622, 144572, 144821, 157485, 171179, 185988, 192713, 195445, 205610, 220733, 224662, 225766]


38

##### Check the result of filling missing value

In [25]:
extracted_rows = []

for index in filled_indices:
    #if index in df.index and (index - 1) in df.index:
    extracted_row = df.loc[[index - 1, index]]
    extracted_rows.append(extracted_row)

extracted_rows_df = pd.concat(extracted_rows)
extracted_rows_df.to_csv('extracted_rows.csv', index=False)

In [26]:
df['Amount'].isnull().sum()

163

### According to two Balance to calculate Amount

In [27]:
df['Amount'].isnull().sum()

163

#### Check the calculate amount inside the mean+-3*std or not, if not use mean, if std is NA or 0 still use mean
#### For personal transfers, use the overall data for filling in missing values. 
#### For merchant transactions, use the average value for each account to fill in the gaps

In [28]:
# Calculate stats for merchant transactions and personal transfers
merchant_stats = df[(df['Third Party Name'].notnull()) & (df['Amount'].notnull())].groupby(['Account No', 'Third Party Name'])['Amount'].agg(['mean', 'std'])
personal_stats = df[(df['Third Party Account No'].notnull()) & (df['Amount'].notnull())].groupby(['Third Party Account No'])['Amount'].agg(['mean', 'std'])

count0, count1, count2 = [0, 0, 0]
count3, count4, count5 = [0, 0, 0]
index_mer1 = []
index_mer2 = []
index_mer3 = []
index_mer4 = []
df['Calculated_Amount'] = None

for i in df[df['Amount'].isnull()].index:
    account_no = df.at[i, 'Account No']
    third_party_name = df.at[i, 'Third Party Name']
    third_party_account_no = df.at[i, 'Third Party Account No']

    # Calculate the amount
    if df.at[i - 1, 'Account No'] == account_no:  # Ensure the previous record is from the same account
        calc_amount = df.at[i, 'Balance'] - df.at[i - 1, 'Balance']  
        df.at[i, 'Calculated_Amount'] = calc_amount  # Store the calculated amount
    else:
        continue

    # If it's a merchant transaction
    if pd.notnull(third_party_name) and pd.isnull(third_party_account_no):
        if (account_no, third_party_name) in merchant_stats.index:
            mean = merchant_stats.loc[(account_no, third_party_name), 'mean']
            std = merchant_stats.loc[(account_no, third_party_name), 'std']
            if pd.notnull(std) and std != 0:
                if (mean - 3*std) <= calc_amount <= (mean + 3*std):
                    df.at[i, 'Amount'] = calc_amount
                    count0 += 1
                else:
                    df.at[i, 'Amount'] = mean
                    index_mer1.append(i)
                    count1 += 1
            else:
                df.at[i, 'Amount'] = mean
                index_mer2.append(i)
                count2 += 1
        
    # If it's a personal transfer
    elif pd.isnull(third_party_name) and pd.notnull(third_party_account_no):
        if third_party_account_no in personal_stats.index:
            mean = personal_stats.loc[(third_party_account_no), 'mean']
            std = personal_stats.loc[(third_party_account_no), 'std']
            if pd.notnull(std) and std != 0:
                if (mean - 3*std) <= calc_amount <= (mean + 3*std):
                    df.at[i, 'Amount'] = calc_amount
                    count3 += 1
                else:
                    df.at[i, 'Amount'] = mean
                    count4 += 1
                    index_mer3.append(i)
            else:
                df.at[i, 'Amount'] = mean
                count5 += 1
                index_mer4.append(i)

# Check the remaining number of records with null Amounts
df['Amount'].isnull().sum(), count0, count1, count2, count3, count4, count5

(2, 145, 5, 11, 0, 0, 0)

#### Check the fill resulte, and manual fix part of them

In [29]:
fill = df.iloc[index_mer1]
fill

,index,Account No,Balance,Amount,Third Party Account No,Third Party Name,DateTime,Calculated_Amount
32249,111098,225210798.0,893.968643,-26.896731,NaN,Cass Art,2023-06-26 10:01:00,-94.97
61533,89764,348325817.0,6841.348691,-17.738333,NaN,Fitted Stitch,2023-05-22 08:33:00,-33.755772
106086,118627,519933553.0,544.003004,-66.868214,NaN,Tesco,2023-07-01 08:34:00,-216.916
123025,26511,583558311.0,557.978785,-73.841905,NaN,Sports Direct,2023-02-01 09:20:00,-177.98
150608,35415,679911053.0,1490.139357,-25.348,NaN,Deliveroo,2023-02-20 00:33:00,-41.24


In [30]:
# Except Tesco
indices_to_update = [32249, 61533, 123025, 150608]
for idx in indices_to_update:
    df.at[idx, 'Amount'] = df.at[idx, 'Calculated_Amount']

fill = df.iloc[index_mer1]
fill

,index,Account No,Balance,Amount,Third Party Account No,Third Party Name,DateTime,Calculated_Amount
32249,111098,225210798.0,893.968643,-94.97,NaN,Cass Art,2023-06-26 10:01:00,-94.97
61533,89764,348325817.0,6841.348691,-33.755772,NaN,Fitted Stitch,2023-05-22 08:33:00,-33.755772
106086,118627,519933553.0,544.003004,-66.868214,NaN,Tesco,2023-07-01 08:34:00,-216.916
123025,26511,583558311.0,557.978785,-177.98,NaN,Sports Direct,2023-02-01 09:20:00,-177.98
150608,35415,679911053.0,1490.139357,-41.24,NaN,Deliveroo,2023-02-20 00:33:00,-41.24


In [31]:
fill = df.iloc[index_mer2]
fill

,index,Account No,Balance,Amount,Third Party Account No,Third Party Name,DateTime,Calculated_Amount
11530,116747,145387690.0,5761.561977,-12.99,NaN,SquareOnix,2023-06-30 23:59:00,-12.99
57305,180285,329840194.0,2039.106730,-12.99,NaN,SquareOnix,2023-09-30 23:59:00,-12.99
59072,202796,337771621.0,-2829.262650,-14.99,NaN,Blizzard,2023-10-31 23:59:00,-14.99
73230,77279,401293808.0,402.612150,-14.99,NaN,Blizzard,2023-04-30 23:59:00,-14.99
79963,1821,424397946.0,3358.010000,-18.99,NaN,PureGym,2023-01-01 00:00:00,-18.99
117315,73954,555804115.0,-1222.221848,-10.04,NaN,JustEat,2023-04-29 02:47:00,-13.34
133373,213902,623156843.0,4600.020616,-17.49,NaN,Etsy,2023-11-17 04:42:00,-54.103313
138371,156819,639549973.0,7060.583110,-14.99,NaN,Blizzard,2023-08-31 23:59:00,-14.99
152655,17105,684988568.0,1155.664625,-99.98,NaN,CeX,2023-01-23 16:41:00,-129.97
203311,224074,887156614.0,-335.438621,-12.99,NaN,SquareOnix,2023-11-30 23:59:00,-12.99


In [32]:
# Change value of JustEat，Etsy，CeX
indices_to_update = [117315, 133373, 152655]
for idx in indices_to_update:
    df.at[idx, 'Amount'] = df.at[idx, 'Calculated_Amount']

fill = df.iloc[index_mer2]
fill

,index,Account No,Balance,Amount,Third Party Account No,Third Party Name,DateTime,Calculated_Amount
11530,116747,145387690.0,5761.561977,-12.99,NaN,SquareOnix,2023-06-30 23:59:00,-12.99
57305,180285,329840194.0,2039.106730,-12.99,NaN,SquareOnix,2023-09-30 23:59:00,-12.99
59072,202796,337771621.0,-2829.262650,-14.99,NaN,Blizzard,2023-10-31 23:59:00,-14.99
73230,77279,401293808.0,402.612150,-14.99,NaN,Blizzard,2023-04-30 23:59:00,-14.99
79963,1821,424397946.0,3358.010000,-18.99,NaN,PureGym,2023-01-01 00:00:00,-18.99
117315,73954,555804115.0,-1222.221848,-13.34,NaN,JustEat,2023-04-29 02:47:00,-13.34
133373,213902,623156843.0,4600.020616,-54.103313,NaN,Etsy,2023-11-17 04:42:00,-54.103313
138371,156819,639549973.0,7060.583110,-14.99,NaN,Blizzard,2023-08-31 23:59:00,-14.99
152655,17105,684988568.0,1155.664625,-129.97,NaN,CeX,2023-01-23 16:41:00,-129.97
203311,224074,887156614.0,-335.438621,-12.99,NaN,SquareOnix,2023-11-30 23:59:00,-12.99


In [33]:
df[df['Amount'].isna()]

,index,Account No,Balance,Amount,Third Party Account No,Third Party Name,DateTime,Calculated_Amount
123247,65949,583579399.0,514.902300,NaN,NaN,NaN,2023-04-08 12:09:00,-50.0346
196979,34681,859264768.0,403.097292,NaN,NaN,CeX,2023-02-18 09:41:00,-79.98


In [34]:
df.at[196979, 'Amount'] = df.at[196979, 'Calculated_Amount']

In [35]:
df.isnull().sum()

index                          0
Account No                   224
Balance                        1
Amount                         1
Third Party Account No    223764
Third Party Name            7079
DateTime                       0
Calculated_Amount         230433
dtype: int64

# Delete missing data

In [36]:
df.isnull().sum()

index                          0
Account No                   224
Balance                        1
Amount                         1
Third Party Account No    223764
Third Party Name            7079
DateTime                       0
Calculated_Amount         230433
dtype: int64

In [37]:
df

,index,Account No,Balance,Amount,Third Party Account No,Third Party Name,DateTime,Calculated_Amount
0,912,101531259.0,124.000000,-859.0,NaN,LBG,2023-01-01 00:00:00,None
1,4036,101531259.0,116.010000,-7.99,NaN,Disney,2023-01-01 00:00:00,None
2,4037,101531259.0,16.010000,-100.0,NaN,Grand Union BJJ,2023-01-01 00:00:00,None
3,4743,101531259.0,-63.990000,-80.0,NaN,Matalan,2023-01-01 09:37:00,None
4,5133,101531259.0,-247.990000,-184.0,NaN,Matalan,2023-01-01 12:00:00,None
...,...,...,...,...,...,...,...,...
230591,223657,NaN,11517.049529,-15.99,NaN,Netflix,2023-11-30 23:59:00,None
230592,226353,NaN,-1692.102105,-15.99,NaN,Netflix,2023-11-30 23:59:00,None
230593,222727,NaN,-731.036300,-1226.0,NaN,Halifax,2023-12-01 00:00:00,None
230594,229119,NaN,2619.340104,-225.37,NaN,Topshop,2023-12-04 10:03:00,None


In [50]:
df = df.dropna(subset=['Account No'])
df = df.dropna(subset=['Third Party Account No', 'Third Party Name'], how='all')
df = df.drop('Calculated_Amount', axis=1)

In [51]:
df.set_index('index', inplace=True)
df.sort_index(inplace=True)
df.reset_index(drop=True, inplace=True)

In [52]:
df

,Account No,Balance,Amount,Third Party Account No,Third Party Name,DateTime
0,678330503.0,2971.000000,1584.0,NaN,Westport Care Home,2023-01-01 00:00:00
1,472213568.0,3792.000000,1950.0,NaN,Barbiee Boutique,2023-01-01 00:00:00
2,472213568.0,3012.000000,-780.0,283027736.0,NaN,2023-01-01 00:00:00
3,283027736.0,1787.000000,780.0,472213568.0,NaN,2023-01-01 00:00:00
4,624500124.0,3226.000000,1825.0,NaN,Fat Face,2023-01-01 00:00:00
...,...,...,...,...,...,...
230121,581655972.0,45935.206861,-41.06,NaN,Tesco,2023-12-06 20:54:00
230122,786141370.0,-244.837500,-62.35,NaN,Sainsbury Local,2023-12-06 20:55:00
230123,824916823.0,9709.172159,-32.94,NaN,Deliveroo,2023-12-06 21:05:00
230124,366550080.0,26834.165794,-19.25,NaN,Amazon,2023-12-06 21:13:00


In [53]:
df.isnull().sum()

Account No                     0
Balance                        0
Amount                         0
Third Party Account No    223297
Third Party Name            6829
DateTime                       0
dtype: int64

In [54]:
df.to_csv('df.csv')